     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=1970fe7f2f2e906d681100337401989603ef52c539a23691eae16014b9329c3a
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *


spark = SparkSession.builder.getOrCreate()

BookSchema = StructType([StructField("year_of_publication", IntegerType(), False),
                       StructField("author", StringType(), False),
                       StructField("name", StringType(), False),
                       StructField("rating", FloatType(), False)])

books_django = [[2020, "William S. Vincent", "Django for Beginners", 9.6], [2022, "William S. Vincent", "Django for APIs", 9.3], [2023, "William S. Vincent", "Django for Professionals", 9.9]]

books_df = spark.createDataFrame(books_django, BookSchema)

books_df.printSchema()
books_df.show()

books_df.drop(books_df.rating).show()

root
 |-- year_of_publication: integer (nullable = false)
 |-- author: string (nullable = false)
 |-- name: string (nullable = false)
 |-- rating: float (nullable = false)
+-------------------+------------------+--------------------+------+
|year_of_publication|            author|                name|rating|
+-------------------+------------------+--------------------+------+
|               2020|William S. Vincent|Django for Beginners|   9.6|
|               2022|William S. Vincent|     Django for APIs|   9.3|
|               2023|William S. Vincent|Django for Profes...|   9.9|
+-------------------+------------------+--------------------+------+
+-------------------+------------------+--------------------+
|year_of_publication|            author|                name|
+-------------------+------------------+--------------------+
|               2020|William S. Vincent|Django for Beginners|
|               2022|William S. Vincent|     Django for APIs|
|               2023|William S. Vin

In [4]:
books_react = [[2018, "John W. Snow", "React for Beginners", 9.3], [2022, "Erick Cartman", "React Cookbook", 8.7], [2023, "Robert Tendy", "Deep Dive in React", 9.0]]
books_react_df = spark.createDataFrame(books_react, BookSchema)
books_react_df.show()

+-------------------+-------------+-------------------+------+
|year_of_publication|       author|               name|rating|
+-------------------+-------------+-------------------+------+
|               2018| John W. Snow|React for Beginners|   9.3|
|               2022|Erick Cartman|     React Cookbook|   8.7|
|               2023| Robert Tendy| Deep Dive in React|   9.0|
+-------------------+-------------+-------------------+------+


In [5]:
new_df = books_df.union(books_react_df)
new_df.show()

+-------------------+------------------+--------------------+------+
|year_of_publication|            author|                name|rating|
+-------------------+------------------+--------------------+------+
|               2020|William S. Vincent|Django for Beginners|   9.6|
|               2022|William S. Vincent|     Django for APIs|   9.3|
|               2023|William S. Vincent|Django for Profes...|   9.9|
|               2018|      John W. Snow| React for Beginners|   9.3|
|               2022|     Erick Cartman|      React Cookbook|   8.7|
|               2023|      Robert Tendy|  Deep Dive in React|   9.0|
+-------------------+------------------+--------------------+------+


In [6]:
new_df.drop('author').show()

+-------------------+--------------------+------+
|year_of_publication|                name|rating|
+-------------------+--------------------+------+
|               2020|Django for Beginners|   9.6|
|               2022|     Django for APIs|   9.3|
|               2023|Django for Profes...|   9.9|
|               2018| React for Beginners|   9.3|
|               2022|      React Cookbook|   8.7|
|               2023|  Deep Dive in React|   9.0|
+-------------------+--------------------+------+


In [7]:
pd_df = new_df.toPandas()
pd_df['read_status'] = [False, True, True, False, True, False]
pd_df

,year_of_publication,author,name,rating,read_status
0,2020,William S. Vincent,Django for Beginners,9.6,False
1,2022,William S. Vincent,Django for APIs,9.3,True
2,2023,William S. Vincent,Django for Professionals,9.9,True
3,2018,John W. Snow,React for Beginners,9.3,False
4,2022,Erick Cartman,React Cookbook,8.7,True
5,2023,Robert Tendy,Deep Dive in React,9.0,False


In [8]:
pd_df.loc[len(pd_df.index)] = [2000, 'Pablo Escobar', 'How to earn million in 7 days', 3, False]


In [9]:
pd_df = pd_df.drop_duplicates(subset=['name'])
pd_df

,year_of_publication,author,name,rating,read_status
0,2020,William S. Vincent,Django for Beginners,9.6,False
1,2022,William S. Vincent,Django for APIs,9.3,True
2,2023,William S. Vincent,Django for Professionals,9.9,True
3,2018,John W. Snow,React for Beginners,9.3,False
4,2022,Erick Cartman,React Cookbook,8.7,True
5,2023,Robert Tendy,Deep Dive in React,9.0,False
6,2000,Pablo Escobar,How to earn million in 7 days,3.0,False


In [10]:
pd_df.drop([len(pd_df.index) - 1], axis='index')

,year_of_publication,author,name,rating,read_status
0,2020,William S. Vincent,Django for Beginners,9.6,False
1,2022,William S. Vincent,Django for APIs,9.3,True
2,2023,William S. Vincent,Django for Professionals,9.9,True
3,2018,John W. Snow,React for Beginners,9.3,False
4,2022,Erick Cartman,React Cookbook,8.7,True
5,2023,Robert Tendy,Deep Dive in React,9.0,False


In [14]:
import pandas as pd


titanic_df = pd.read_csv('/content/Titanic.csv')
titanic_df[:25]

,pclass,survived,Residence,name,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest,Gender
0,3,0,0,"Abbing, Mr. Anthony",42,0,0,C.A. 5547,7.55,,S,,,,0
1,3,0,0,"Abbott, Master. Eugene Joseph",13,0,2,C.A. 2673,20.25,,S,,,"East Providence, RI",0
2,3,0,0,"Abbott, Mr. Rossmore Edward",16,1,1,C.A. 2673,20.25,,S,,190,"East Providence, RI",0
3,3,1,0,"Abbott, Mrs. Stanton (Rosa Hunt)",35,1,1,C.A. 2673,20.25,,S,A,,"East Providence, RI",1
4,3,1,2,"Abelseth, Miss. Karen Marie",16,0,0,348125,7.65,,S,16,,"Norway Los Angeles, CA",1
5,3,1,0,"Abelseth, Mr. Olaus Jorgensen",25,0,0,348122,7.65,F G63,S,A,,"Perkins County, SD",0
6,2,0,2,"Abelson, Mr. Samuel",30,1,0,P/PP 3381,24,,C,,,"Russia New York, NY",0
7,2,1,2,"Abelson, Mrs. Samuel (Hannah Wizosky)",28,1,0,P/PP 3381,24,,C,10,,"Russia New York, NY",1
8,3,1,2,"Abrahamsson, Mr. Abraham August Johannes",20,0,0,SOTON/O2 3101284,7.925,,S,15,,"Taalintehdas, Finland Hoboken, NJ",0
9,3,1,2,"Abrahim, Mrs. Joseph (Sophie Halaut Easu)",18,0,0,2657,7.2292,,C,C,,"Greensburg, PA",1


In [15]:
titanic_df['age'] = pd.to_numeric(titanic_df['age'], errors='coerce', downcast='integer')
children = titanic_df[titanic_df['age'] < 18]
len(children)

154

In [16]:
sw3 = titanic_df[(titanic_df['Gender'] == 1) & (titanic_df['pclass'] == 3) & (titanic_df['survived'] == 1)]
sw3

,pclass,survived,Residence,name,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest,Gender
3,3,1,0,"Abbott, Mrs. Stanton (Rosa Hunt)",35.0,1,1,C.A. 2673,20.25,,S,A,,"East Providence, RI",1
4,3,1,2,"Abelseth, Miss. Karen Marie",16.0,0,0,348125,7.65,,S,16,,"Norway Los Angeles, CA",1
9,3,1,2,"Abrahim, Mrs. Joseph (Sophie Halaut Easu)",18.0,0,0,2657,7.2292,,C,C,,"Greensburg, PA",1
14,3,1,2,"Aks, Mrs. Sam (Leah Rosen)",18.0,0,1,392091,9.35,,S,13,,"London, England Norfolk, VA",1
28,3,1,2,"Andersen-Jensen, Miss. Carla Christine Nielsine",19.0,1,0,350046,7.8542,,S,16,,,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1210,3,1,2,"Turja, Miss. Anna Sofia",18.0,0,0,4138,9.8417,,S,15,,,1
1211,3,1,2,"Turkula, Mrs. (Hedwig)",63.0,0,0,4134,9.5875,,S,15,,,1
1263,3,1,2,"Whabee, Mrs. George Joseph (Shawneene Abi-Saab)",38.0,0,0,2688,7.2292,,C,C,,,1
1279,3,1,1,"Wilkes, Mrs. James (Ellen Needs)",47.0,1,0,363272,7,,S,,,,1


In [17]:
titanic_df['fare'] = pd.to_numeric(titanic_df['fare'], errors='coerce')
americans = titanic_df[titanic_df['Residence'] == 0]
total = americans['fare'].sum()
total

16680.5

In [18]:
titanic_df.sort_values('age')[100::30]

,pclass,survived,Residence,name,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest,Gender
1172,2,0,1,"Sweet, Mr. George Frederick",14.0,0,0,220845,65.0000,,S,,,"Somerset / Bernardsville, NJ",0
467,3,0,1,"Goodwin, Miss. Lillian Amy",16.0,5,2,CA 2144,46.9000,,S,,,"Wiltshire, England Niagara Falls, NY",1
344,2,1,1,"Doling, Miss. Elsie",18.0,0,1,231919,23.0000,,S,,,Southampton,1
392,3,0,2,"Fischer, Mr. Eberhard Thelander",18.0,0,0,350036,7.7958,,S,,,,0
347,3,1,1,"Dorking, Mr. Edward Arthur",19.0,0,0,A/5. 10482,8.0500,,S,B,,"England Oglesby, IL",0
566,2,0,1,"Hood, Mr. Ambrose Jr",21.0,0,0,S.O.C. 14879,73.5000,,S,,,"New Forest, England",0
446,2,0,1,"Giles, Mr. Edgar",21.0,1,0,28133,11.5000,,S,,,"Cornwall / Camden, NJ",0
265,2,1,2,"Cook, Mrs. (Selena Rogers)",22.0,0,0,W./C. 14266,10.5000,F33,S,14,,Pennsylvania,1
370,2,0,1,"Eitemiller, Mr. George Floyd",23.0,0,0,29751,13.0000,,S,,,"England / Detroit, MI",0
980,3,0,2,"Pokrnic, Mr. Tome",24.0,0,0,315092,8.6625,,S,,,,0


In [29]:
from pyspark.sql import SparkSession


spark = SparkSession.builder.getOrCreate()


df = spark.read.csv("/content/Titanic.csv", header=True, inferSchema=True)

df = df.withColumn("age", df["age"].cast(IntegerType()))


df.createOrReplaceTempView("titanic")


children_count = spark.sql("SELECT COUNT(name) FROM titanic WHERE age < 18")


children_count.show()

+-----------+
|count(name)|
+-----------+
|        154|
+-----------+


In [30]:
sw = spark.sql("SELECT name FROM titanic WHERE Gender == 1 AND pclass == 3 AND survived == 1")

sw.show()

+--------------------+
|                name|
+--------------------+
|Abbott, Mrs. Stan...|
|Abelseth, Miss. K...|
|Abrahim, Mrs. Jos...|
|Aks, Mrs. Sam (Le...|
|Andersen-Jensen, ...|
|Andersson, Miss. ...|
|Asplund, Miss. Li...|
|Asplund, Mrs. Car...|
|"Assaf Khalil, Mr...|
|Ayoub, Miss. Banoura|
|Backstrom, Mrs. K...|
|Baclini, Miss. Eu...|
|Baclini, Miss. He...|
|Baclini, Miss. Ma...|
|Baclini, Mrs. Sol...|
|Badman, Miss. Emi...|
|Bradley, Miss. Br...|
|"Carr, Miss. Hele...|
|Connolly, Miss. Kate|
|"Coutts, Mrs. Wil...|
+--------------------+


In [31]:
americans_spent = spark.sql("SELECT SUM(fare) FROM titanic WHERE Residence == 0")

americans_spent.show()

+------------------+
|         sum(fare)|
+------------------+
|16680.499999999985|
+------------------+


In [32]:
desc_order = spark.sql("SELECT * FROM titanic ORDER BY age DESC")

desc_order.show()

+------+--------+---------+--------------------+---+-----+-----+----------+--------+-----------+--------+----+----+--------------------+------+
|pclass|survived|Residence|                name|age|sibsp|parch|    ticket|    fare|      cabin|embarked|boat|body|           home.dest|Gender|
+------+--------+---------+--------------------+---+-----+-----+----------+--------+-----------+--------+----+----+--------------------+------+
|     1|       1|        1|Barkworth, Mr. Al...| 80|    0|    0|     27042|      30|        A23|       S|   B|    |       Hessle, Yorks|     0|
|     1|       1|        1|Cavendish, Mrs. T...| 76|    1|    0|     19877|   78.85|        C46|       S|   6|    |Little Onn Hall, ...|     1|
|     3|       0|        2| Svensson, Mr. Johan| 74|    0|    0|    347060|   7.775|           |       S|    |    |                    |     0|
|     1|       0|        2|Artagaveytia, Mr....| 71|    0|    0|  PC 17609| 49.5042|           |       C|    |  22| Montevideo, Uruguay|